# Import libraries

In [1]:
import requests
import pandas as pd

# Extract, Transform, Load (ETL)

## Extract Data


In [4]:
url = "https://datahub.io/core/top-level-domain-names/r/top-level-domain-names.csv.csv"

# Send a GET request to the URL
response = requests.get(url)

# Check first if the request was successful 
# Write the content to a file if successful
if response.status_code == 200:
    with open("data.csv", "wb") as file:
        file.write(response.content)
    print("Success! File downloaded")
else:
    print("Failed to download file:", response.status_code)


Success! File downloaded
